In [ ]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(11,df.shape[0]-1):
        sugg = 0
        if df[each+1][3] > df[each][3]:
          sugg = 1
        row = [
                df[each-10][3],
                df[each-9][3],
                df[each-8][3],
                df[each-7][3],
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                df[each][3], 
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["sugg"] = last_column
  return scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df      

In [ ]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")

symbols = get_crypto_syms()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
data = process(dfs)
data.to_csv("1d_6pc_10mms.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data

In [ ]:
symbols = read_syms_from_txt()
len(symbols)
pieces = 15
new_arrays = np.array_split(symbols, pieces)
for i in new_arrays:
  dfs = []
  for x in i:
        data = yf.download(x,period="MAX",interval="1d",progress=False)
        if data.empty :
             print("Passing...")
        else:
               dfs.append(data)
  ndfs = dfs
  dfs = []
  for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
  data = process(dfs)
  data.to_csv(f"{x}.csv")         

In [9]:
import os
files = os.listdir("/content/gdrive/MyDrive/Colab Files/")

data = pd.DataFrame()
for file in files:
  address = f"/content/gdrive/MyDrive/Colab Files/{file}"
  df = pd.read_csv(address)
  data = pd.concat([data,df])

In [ ]:
xTrain, xTest, yTrain, yTest = spliting(data)
data

In [ ]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=3e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.add(Dense(1500, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(1000, activation='relu'))
model.add(Dense(750, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=100,batch_size=50000,validation_data=(xTest,yTest))
model.evaluate(xTest,yTest)
model.save(f"{model.evaluate(xTest,yTest)}.h5")

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")